In [1]:
from pydantic import BaseModel, Field
from langchain.prompts import ChatPromptTemplate

from typing import List
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser

In [4]:
def a():
    file_name = "example_data.csv"
    file_format = "CSV"
    summary_str = "This file contains data related to sales transactions, including customer information, product details, and transaction amounts."
    questions_str = "1. What are the top 5 products by sales volume?\n2. How does customer age correlate with purchase frequency?\n3. What is the average transaction amount per customer?\n4. Are there any seasonal trends in sales data?"

    final_summary = """## File Summary:\n\n#### File Name: {}\n#### File Format: {}\n#### Description:\n{}\n\n## Here are some questions that can be asked for EDA:\n{}.""".format(
        file_name if file_name else "N/A",
        file_format if file_format else "N/A",
        summary_str,
        questions_str
    )
    return final_summary


# Define the file path (you can change the name or path as needed)
file_path = "./output_summary.txt"

summ = a()
# Write the text to the file
with open(file_path, "w", encoding="utf-8") as file:
    file.write(summ)

# Return the file path (optional)
print(f"Text written to: {file_path}")

Text written to: ./output_summary.txt
